In [8]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

In [9]:
all_rows = pd.DataFrame(columns=['phrase', 'is_ingredient'])

In [10]:
for x in tqdm(range(200,700)):
    page = requests.get("http://www.geniuskitchen.com/recipe/a-" + str(x))
    if(page.status_code == 404):
        continue
    soup = BeautifulSoup(page.content, 'html.parser')
    ingrs_str = [i.get_text() for i in soup.select('ul.ingredient-list')][0]
    ingrs = [s for s in ingrs_str.split('\n') if s]
    title = soup.select('h1')[0].get_text()
    directions = [d.get_text() for d in soup.select('.directions-inner li')]
    df_ingr = pd.DataFrame({'phrase':ingrs})
    df_ingr['is_ingredient'] = 1
    df_title = pd.DataFrame({'phrase':[title]})
    df_title['is_ingredient'] = 0
    df_directions = pd.DataFrame({'phrase':directions})
    df_directions['is_ingredient'] = 0
    all_rows = pd.concat([all_rows, df_ingr, df_title, df_directions], ignore_index=1)

100%|██████████| 500/500 [04:44<00:00,  1.76it/s]


In [11]:
all_rows.describe()


,phrase,is_ingredient
count,9285,9285
unique,7487,2
top,\n\n\n\n\n\n\n\n \n...,0
freq,458,5108


In [12]:
all_rows['phrase'] = all_rows['phrase'].str.lower()
all_rows.head()

,phrase,is_ingredient
0,"4 medium chicken breast halves (skinless, bo...",1
1,salt and pepper,1
2,1⁄4 cup lemon juice,1
3,2 tablespoons lime juice,1
4,2 tablespoons orange juice,1


In [13]:
all_rows.to_csv('../data/recipe_phrases.csv', index=False)
